In [ ]:
import import_ipynb

In [ ]:
from SoilWaterFlow import *
import Conversion as conv
import DistanceMap as dis
import ChannelFlow as chn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as mt

In [ ]:
max_GWnrIterations = 2
tolerance = 0.01 
mag_factor = 5

## Aquifer parameters
Ne = 0.0557533 ## Unit: m3/m3 (Effective porosity, Specific yield)
Ks = 8.411E-04 ## Unit should be m/s (note, the unit of Ks in soil is kgs/m3)

## Experimental variable
dist_to_zeroBC = 4

Kr = Ks * 3600 ## River bottom sediment conductivity, m/hr
Mr = (dis.resolution / mag_factor) # River bottom sediment thickness, m

Cat_grid_size = dis.resolution # Catchment grid size (unit: m)
Qlateral = np.zeros(10000)
Wp = np.zeros(10000)

In [ ]:
## Varied-in-space Ksat 
# Ks = pd.read_csv('Data/Ks-experiment.csv', header=0, parse_dates=[0], index_col=0)

In [ ]:
def WT_Dynamics(GWL_head, Averaged_SH, tmax, xmax, WT_initial, FrameNumber):
    
    ## Courant number= a(dt/dx)
    dt = tmax
    dx = Cat_grid_size / mag_factor ## dx: 50 m     

    h = np.zeros(xmax*mag_factor)
    x = np.zeros(xmax*mag_factor)    

    ## Initializing GWL 
    GW_sum0 = 0
    if FrameNumber <= 1 :
        for i in range(1, xmax*mag_factor):
            if i < (xmax*mag_factor)-mag_factor:
                h[i] = WT_initial
            elif i >= (xmax*mag_factor)-mag_factor:
                h[i] = WT_initial 
            GW_sum0 += (h[i] * pow(dx, 2)) * Ne     
    
    elif FrameNumber > 1:
        for i in range(1, xmax*mag_factor):
            if i < (xmax*mag_factor)-mag_factor:
                h[i] = GWL_head[mt.ceil(i/mag_factor)]
            elif i >= (xmax*mag_factor)-mag_factor:
                h[i] = GWL_head[mt.ceil(i/mag_factor)-1] 
            GW_sum0 += (h[i] * pow(dx, 2)) * Ne 
    
    ## Wetted perimeter according to (averaged) river stage (unit: m)
#     Wp[FrameNumber] = ((chn.Bw + 2 * pow((pow(Averaged_SH, 2) + pow(chn.z, 2)), 0.5)) / 2.0)
    Wp[FrameNumber] = ((chn.Bw + 2 * Averaged_SH) / 2.0)
#     Wp[FrameNumber] = Averaged_SH

    ## Time-dependent river stage boundary condition    
    h[0] = Averaged_SH
#     ## Constant head B.C. 
#     h[xmax*mag_factor-1, :] = 10.0

    GW_massbalance = GW_sum0
    GW_nrIteration = 0

    while ((GW_massbalance > (GW_sum0 * tolerance)) and (GW_nrIteration < max_GWnrIterations)):
        for i in range(1, xmax*mag_factor-1):
            ## Constant-in-space Ks
            h[i] = (dt / dx) * (Ks / Ne) * (h[i] * ((h[i+1] - h[i]) / dx)\
                                                              - h[i-1] * ((h[i] -\
                                                                h[i-1]) / dx)) + h[i]         
        
#         ## Zero flux river stage boundary condition - No Flow between SW and GW  
#         h[0, t+1] = h[1, t+1] 
        ## Zero flux boundary condition - the edge of watershed
        h[xmax*mag_factor-1] = h[xmax*mag_factor-2] 

        GW_newSum = 0
        for i in range(1, xmax*mag_factor):
            GW_newSum += (h[i] * pow(dx, 2)) * Ne 
            
        ## Unlike Soil, GW_newSum = m3, GW_sum0 = m3,  = kg
        GW_massbalance = abs(GW_newSum - (GW_sum0 -\
                                          (-Kr * (Wp[FrameNumber] * Cat_grid_size) * (h[0] - h[1]) * (1 / Mr))))
        GW_nrIteration += 1  
  
    #in case: (xmax * mag_factor-1) = dist_to_zeroBC
#     while ((GW_massbalance > (GW_sum0 * tolerance)) and (GW_nrIteration < max_GWnrIterations)):
#         for i in range(1, dist_to_zeroBC):
#             ## Constant-in-space Ks
#             h[i] = (dt / dx) * (Ks / Ne) * (h[i] * ((h[i+1] - h[i]) / dx)\
#                                                               - h[i-1] * ((h[i] -\
#                                                                 h[i-1]) / dx)) + h[i]         
        
# #         ## Zero flux river stage boundary condition - No Flow between SW and GW  
# #         h[0, t+1] = h[1, t+1] 
#         ## Zero flux boundary condition - the edge of watershed
#         h[dist_to_zeroBC] = h[dist_to_zeroBC-1] 

#         GW_newSum = 0
#         for i in range(1, xmax*mag_factor):
#             GW_newSum += (h[i] * pow(dx, 2)) * Ne 
            
#         ## Unlike Soil, GW_newSum = m3, GW_sum0 = m3,  = kg
#         GW_massbalance = abs(GW_newSum - (GW_sum0 -\
#                                           (-Kr * (Wp[FrameNumber] * Cat_grid_size) * (h[0] - h[1]) * (1 / Mr))))
#         GW_nrIteration += 1  
    
    ## Lateral drainage
    if (GW_massbalance < tolerance):

        if h[1] > (-Mr):
            Qlateral[FrameNumber] = (-Kr * (Wp[FrameNumber] * Cat_grid_size)\
                                     * (h[0] - h[1]) * (1 / Mr))  
            
        elif h[1] <= (-Mr):
            Qlateral[FrameNumber] = (-Kr * (Wp[FrameNumber] * Cat_grid_size) * ((Averaged_SH - (-Mr)) / Mr)) 
        
        return h[:], x, Qlateral
    
    elif (GW_massbalance >= tolerance):

        if h[1] > (-Mr):
            Qlateral[FrameNumber] = (-Kr * (Wp[FrameNumber] * Cat_grid_size)\
                                     * (h[0] - h[1]) * (1 / Mr))  
            
        elif h[1] <= (-Mr):
            Qlateral[FrameNumber] = (-Kr * (Wp[FrameNumber] * Cat_grid_size) * ((Averaged_SH - (-Mr)) / Mr)) 
        
        return h[:], x, Qlateral